In [1]:
deployment = "gpt4"
model = "gpt-4"

In [3]:
def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [5]:
import openai
import json


funcs = {"get_current_cluster_state": get_current_cluster_state}


def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret["content"] 

def run_conversation(msg):
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=msg,
        functions=[
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster",
                        },
                        
                    },
                    "required": ["cluster_name"],
                },
            }
        ],
        function_call="auto",
    )

    message =  response["choices"][0]["message"]
    print("----- message ----")
    print(message)
    print("----- message ----")
    # 如果不需要调用function，则直接返回结果
    if not message.get("function_call"):
        return message
    
    # 获取调用的方法
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    print(function_args)
    res = funcs[function_name](function_args["cluster_name"]) 
    message["content"]=None
    msg.append(message)
    msg.append({
                "role": "function",
                "name": function_name,
                "content": res,
            })
    return run_conversation(msg)
   

In [6]:
#run("Hello")
print(run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. "))

----- message ----
{
  "role": "assistant",
  "function_call": {
    "name": "get_current_cluster_state",
    "arguments": "{\n  \"cluster_name\": \"DSP\"\n}"
  }
}
----- message ----
{'cluster_name': 'DSP'}
cluster:DSP
----- message ----
{
  "role": "assistant",
  "content": "The current issue with the 'DSP' cluster seems to be related to Docker. Specifically, it is unable to pull the image \"chaocai/docker/dsp:latest\". \n\nThis issue could be caused by several reasons:\n\n1. There might be network issues preventing pulling the Docker image.\n2. The image might not exist or it's not accessible. Please validate whether the image \"chaocai/docker/dsp:latest\" is actually present and accessible.\n3. Docker daemon might be down or not working properly.\n\nFollowing are some suggestions to tackle this issue:\n\n1. Check your network connection and try again.\n2. Check if the Docker image \"chaocai/docker/dsp:latest\" exists and is accessible.\n3. Check the Docker daemon, restart it if nec